In [1]:
import tensorflow as tf
import pandas as pd
import pickle
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model= load_model('model.h5')

In [3]:
with open('label_encoder_gender.pkl','rb') as file:
    gender_label_encoder=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler= pickle.load(file)

In [8]:
input_data={
    "CreditScore":600,
    "Geography":'France',
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [12]:
geo_encoded= onehot_geo.transform([[input_data['Geography']]]).toarray()

onehot_geo.get_feature_names_out()

geo_df= pd.DataFrame(geo_encoded,columns=onehot_geo.get_feature_names_out())

geo_df

d:\GenAI\25122024\tf\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [25]:
input_df['Gender']=gender_label_encoder.transform(input_df['Gender'])

In [26]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [27]:
input_df= pd.concat([input_df.drop('Geography',axis=1),geo_df],axis=1)

In [28]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [29]:
# Scaling the Data
input_scale= scaler.transform(input_df)
input_scale

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
prediction= model.predict(input_scale)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


array([[0.0290434]], dtype=float32)

In [31]:
prediction_prob= prediction[0][0]

In [32]:
if prediction_prob > 0.5:
    print("The customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
